In [1]:
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim

# Dataset Class
class MarineDebrisDataset(Dataset):
    def __init__(self, folderPath, transform=None):
        self.folderPath = folderPath
        self.fileNames = [f for f in os.listdir(folderPath) if f.endswith('.pkl')]
        self.transform = transform

    def __len__(self):
        return len(self.fileNames)

    def __getitem__(self, idx):
        filePath = os.path.join(self.folderPath, self.fileNames[idx])
        with open(filePath, 'rb') as file:
            data = pickle.load(file)
        
        image = data[0]
        label = data[1]
        
        return image, label

# Data Augmentation and Transformations
transformTrain = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 12, std=[0.2] * 12)
])

transformTest = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5] * 12, std=[0.2] * 12)
])

# Initialize datasets
trainDataset = MarineDebrisDataset(folderPath="classification_dataset/train", transform=transformTrain)
valDataset = MarineDebrisDataset(folderPath="classification_dataset/validation", transform=transformTest)
testDataset = MarineDebrisDataset(folderPath="classification_dataset/test", transform=transformTest)

# Handle class imbalance
trainLabels = []
for _, label in trainDataset:
    trainLabels.append(int(label))

classCounts = np.bincount(trainLabels)
classWeights = 1.0 / classCounts
sampleWeights = [classWeights[label] for label in trainLabels]
sampler = WeightedRandomSampler(sampleWeights, len(trainDataset))

# DataLoaders
trainLoader = DataLoader(trainDataset, batch_size=32, sampler=sampler)
valLoader = DataLoader(valDataset, batch_size=32, shuffle=False)
testLoader = DataLoader(testDataset, batch_size=32, shuffle=False)
print("Loaded data")
# Model Definition
class MarineDebrisClassifier(nn.Module):
    def __init__(self, numBands):
        super(MarineDebrisClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=numBands, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = nn.functional.adaptive_avg_pool2d(x, (4, 4))  # Downsample feature map
        x = x.view(-1, 256 * 4 * 4)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)  # Output layer
        return x

# Training Function
def trainModel(model, trainLoader, valLoader, numEpochs, learningRate):
    print("Training Started...")
    criterion = nn.BCELoss()  # Binary cross-entropy
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    bestValLoss = float('inf')
    
    for epoch in range(numEpochs):
        model.train()
        runningLoss = 0.0
        for inputs, labels in trainLoader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))  # Apply sigmoid, labels expect single-channel
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
        
        model.eval()
        valLoss = 0.0
        with torch.no_grad():
            for inputs, labels in valLoader:
                inputs, labels = inputs.to(device), labels.to(device).float()
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                valLoss += loss.item()

        print(f"Epoch [{epoch + 1}/{numEpochs}], Train Loss: {runningLoss/len(trainLoader):.4f}, Val Loss: {valLoss/len(valLoader):.4f}")

        # Save the best model based on validation loss
        if valLoss < bestValLoss:
            bestValLoss = valLoss
            torch.save(model.state_dict(), 'best_model.pth')

# Inference Function
def inference(model, testLoader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in testLoader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.squeeze().cpu().numpy())
    return np.array(predictions)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Main script
if __name__ == "__main__":
    model = MarineDebrisClassifier(numBands=12).to(device)
    trainModel(model, trainLoader, valLoader, numEpochs=10, learningRate=1e-3)
    testPredictions = inference(model, testLoader)
    print(f"Sample Predictions: {testPredictions[:10]}")


Loaded data
Training Started...
Epoch [1/10], Train Loss: 0.3629, Val Loss: 0.3171
Epoch [2/10], Train Loss: 0.2935, Val Loss: 0.2839
Epoch [3/10], Train Loss: 0.2610, Val Loss: 0.2515
Epoch [4/10], Train Loss: 0.2428, Val Loss: 0.2414
Epoch [5/10], Train Loss: 0.2274, Val Loss: 0.2649


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import torch
import numpy as np

def evaluateModel(model, dataLoader, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.eval()
    allPredictions = []
    allLabels = []

    with torch.no_grad():
        for images, labels in dataLoader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            
            # Collect predictions and labels
            allPredictions.extend(predictions.cpu().numpy())
            allLabels.extend(labels.cpu().numpy())
    
    # Check if all classes are present in predictions
    predictedClasses = set(allPredictions)
    trueClasses = set(allLabels)
    missingClasses = trueClasses - predictedClasses
    if missingClasses:
        print(f"Warning: The model did not predict the following classes: {missingClasses}")
    
    # Compute metrics with zero_division set to 0
    print("Classification Report:")
    print(classification_report(
        allLabels, 
        allPredictions, 
        target_names=["No Debris", "Debris"], 
        zero_division=0
    ))
    
    # Confusion matrix
    cm = confusion_matrix(allLabels, allPredictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Debris", "Debris"])
    disp.plot(cmap="Blues", values_format="d")


# Assuming 'model' is your trained model
evaluateModel(model, testLoader)

